In [ ]:
import os
import numpy as np


from dotenv import load_dotenv

from sqlalchemy import func, text
from sqlalchemy.orm import Session

from typing import List, Tuple, Dict

from fcb_data_providers.database_models import Event, Qualifier, Player, Team, Match
from fcb_data_providers.database import Database

load_dotenv()

In [ ]:
DATABASE_URL=os.getenv("DATABASE_URL")
DATA_DIR=os.getenv("DATA_DIR")
db = Database(DATABASE_URL)
session = db.get_session() 

In [ ]:
providers = StatsPerformProvider(data_path=DATA_DIR, database_url=DATABASE_URL)

In [ ]:
providers.process_data()

In [3]:
import sys
sys.path.insert(0, "..")

In [4]:
from common.data_provider import get_match_details_from_view_by_match_id

In [ ]:
get_match_details_from_view_by_match_id(session, 'bt9ewry54yyw87kdh2l9p4vmc')

In [19]:
match_id = 'bt9ewry54yyw87kdh2l9p4vmc'
query = text(f"""select * from match_details where id = '{match_id}';""")
# Execute raw SQL query using session.execute()
query_result = session.execute(query).fetchone()

# Create a dictionary from the query results
match_details = {
    "match_id": query_result[0],
    "match_date": query_result[1],
    "match_status": query_result[2],
    "teams": [
        {"team_name": query_result[3], "team_id": query_result[4]},
        {"team_name": query_result[5], "team_id": query_result[6]},
    ],
    "winner": query_result[7],
    "match_length_min": query_result[8],
    "match_length_sec": query_result[9],
}

In [16]:
from logging import Logger

from sqlalchemy import text
from sqlalchemy.orm.session import Session


CREATE_MATCH_DETAIL_VIEW_QUERY = text(
    """
        CREATE OR REPLACE VIEW match_details AS
        SELECT
            m.id,
            m.match_date,
            m.match_status,
            ht.official_name AS home_team_name,
            m.home_team_id,
            at.official_name AS away_team_name,
            m.away_team_id,
            m.winner,
            m.match_length_min,
            m.match_length_sec
        FROM
            matches m
        JOIN
            teams ht ON (m.home_team_id = ht.id)
        JOIN
            teams at ON (m.away_team_id = at.id);
    """)

In [ ]:

try:
    session.execute(CREATE_MATCH_DETAIL_VIEW_QUERY)
    session.commit()
    print("Match details view created successfully")
    
except Exception as exc:
    session.rollback()
    print(f"Error while creating match details view: {exc}")

In [ ]:
team_id = "apoawtpvac4zqlancmvw4nk4o"

In [ ]:
def get_all_passes_by_players_count(session: Session, team_id: str) -> List[Tuple]:
    """
    Retrieving all passes by players count

    :param session: Session: Session manager to get the data from database of your choice.
    :param team_id: str: Team ID
    return: List[tuple]: List of tuples, which contain player name and KPI value count.
    """
    result = (
        session.query(Player.match_name, func.count(Event.player_id))
        .join(Qualifier, Event.e_id == Qualifier.event_id)
        .join(Player, Event.player_id == Player.id)
        .filter(
            Event.type_id == 1,
            Event.team_id == team_id,
            ~Qualifier.qualifier_id.in_([2, 107, 123]),
        )
        .group_by(Player.match_name)
        .order_by(Player.match_name)
        .all()
    )

    return result

In [ ]:
get_all_passes_by_players_count(session, team_id)

In [ ]:

def get_successfull_passes_by_players_count(
    session: Session, team_id: str
) -> List[Tuple]:
    """
    Retrieving all passes by players count

    :param session: Session: Session manager to get the data from database of your choice.
    :param team_id: str: Team ID
    return: List[tuple]: List of tuples, which contain player name and KPI value count.
    """
    result = (
        session.query(Player.match_name, func.count(Event.player_id))
        .join(Qualifier, Event.e_id == Qualifier.event_id)
        .join(Player, Event.player_id == Player.id)
        .filter(
            Event.type_id == 1,
            Event.outcome == "1",
            Event.team_id == team_id,
            ~Qualifier.qualifier_id.in_([2, 107, 123]),
        )
        .group_by(Player.match_name)
        .order_by(Player.match_name)
        .all()
    )

    return result

In [ ]:
get_successfull_passes_by_players_count(session, team_id)

In [ ]:
def get_key_passes_by_players_count(session: Session, team_id: str) -> List[Tuple]:
    """
    Retrieving all passes by players count

    :param session: Session: Session manager to get the data from database of your choice.
    :param team_id: str: Team ID
    return: List[tuple]: List of tuples, which contain player name and KPI value count.
    """
    result = (
        session.query(Player.match_name, func.count(Event.player_id))
        .join(Qualifier, Event.e_id == Qualifier.event_id)
        .join(Player, Event.player_id == Player.id)
        .filter(
            Event.type_id.in_([13, 14, 15, 60]),  # Multiple type_id values
            Event.team_id == team_id,
            Qualifier.qualifier_id.in_([29, 55]),  # Specific qualifier_id values
        )
        .group_by(Player.match_name)
        .order_by(Player.match_name)
        .all()
    )

    return result

In [ ]:
get_key_passes_by_players_count(session, team_id)

In [ ]:
def get_long_passes_by_players_count(session: Session, team_id: str) -> List[Tuple]:
    """
    Retrieving all passes by players count

    :param session: Session: Session manager to get the data from database of your choice.
    :param team_id: str: Team ID
    return: List[tuple]: List of tuples, which contain player name and KPI value count.
    """
    result = (
        session.query(Player.match_name, func.count(Event.player_id))
        .join(Qualifier, Event.e_id == Qualifier.event_id)
        .join(Player, Event.player_id == Player.id)
        .filter(
            Event.type_id == 1,  # Specific event type
            Event.team_id == team_id,  # Specific team ID
            Qualifier.qualifier_id == 1,  # Specific qualifier ID
            ~Qualifier.qualifier_id.in_(
                [2, 107, 123]
            ),  # Exclusion of certain qualifier IDs
        )
        .group_by(Player.match_name)
        .order_by(Player.match_name)
        .all()
    )

    return result

In [ ]:
get_long_passes_by_players_count(session, team_id)

In [ ]:
def get_shots_on_goals_by_players_count(session: Session, team_id: str) -> List[Tuple]:
    """
    Retrieving all passes by players count

    :param session: Session: Session manager to get the data from database of your choice.
    :param team_id: str: Team ID
    return: List[tuple]: List of tuples, which contain player name and KPI value count.
    """
    result = (
        session.query(Player.match_name, func.count(Event.player_id))
        .join(Qualifier, Event.e_id == Qualifier.event_id)
        .join(Player, Event.player_id == Player.id)
        .filter(
            Event.type_id.in_([13, 14, 15, 16]),  # Multiple event types
            Event.team_id == team_id,  # Specific team ID
        )
        .group_by(Player.match_name)
        .order_by(Player.match_name)
        .all()
    )

    return result

In [ ]:
get_shots_on_goals_by_players_count(session, team_id)

In [ ]:
def get_all_aerials_duels_by_players_count(
    session: Session, team_id: str
) -> List[Tuple]:
    """
    Retrieving all passes by players count

    :param session: Session: Session manager to get the data from database of your choice.
    :param team_id: str: Team ID
    return: List[tuple]: List of tuples, which contain player name and KPI value count.
    """
    result = (
        session.query(Player.match_name, func.count(Event.player_id))
        .join(Qualifier, Event.e_id == Qualifier.event_id)
        .join(Player, Event.player_id == Player.id)
        .filter(
            Event.type_id == 4,  # Specific event type
            Event.team_id == team_id,  # Specific team ID
        )
        .group_by(Player.match_name)
        .order_by(Player.match_name)
        .all()
    )

    return result

In [ ]:
get_all_aerials_duels_by_players_count(session, team_id)

In [ ]:
def get_successfull_aerials_duels_by_players_count(
    session: Session, team_id: str
) -> List[Tuple]:
    """
    Retrieving all passes by players count

    :param session: Session: Session manager to get the data from database of your choice.
    :param team_id: str: Team ID
    return: List[tuple]: List of tuples, which contain player name and KPI value count.
    """
    result = (
        session.query(Player.match_name, func.count(Event.player_id))
        .join(Qualifier, Event.e_id == Qualifier.event_id)
        .join(Player, Event.player_id == Player.id)
        .filter(
            Event.type_id == 4,  # Specific event type
            Event.team_id == team_id,  # Specific team ID
            Event.outcome == "1",  # Specific outcome condition
        )
        .group_by(Player.match_name)
        .order_by(Player.match_name)
        .all()
    )

    return result

In [ ]:
get_successfull_aerials_duels_by_players_count(session, team_id)

In [ ]:
def get_all_metrics(session: Session, team_id: str) -> Dict[str, List[int]]:
    """
    Retrieve and combine various player statistics for a given team.

    Args:
        session: SQLAlchemy session
        team_id: Team ID
    """
    # Retrieve statistics
    all_passes = get_all_passes_by_players_count(session, team_id)
    successful_passes = get_successfull_passes_by_players_count(session, team_id)
    key_passes = get_key_passes_by_players_count(session, team_id)
    long_passes = get_long_passes_by_players_count(session, team_id)
    shots_on_goal = get_shots_on_goals_by_players_count(session, team_id)
    all_aerial_duels = get_all_aerials_duels_by_players_count(session, team_id)
    successful_aerial_duels = get_successfull_aerials_duels_by_players_count(
        session, team_id
    )

    return (
        all_passes,
        successful_passes,
        key_passes,
        long_passes,
        shots_on_goal,
        all_aerial_duels,
        successful_aerial_duels,
    )

In [ ]:
(
    all_passes,
    successful_passes,
    key_passes,
    long_passes,
    shots_on_goal,
    all_aerial_duels,
    successful_aerial_duels,
) = get_all_metrics(session, team_id)

In [ ]:
all_passes

In [ ]:

def z_score_normalize_with_scaling(
    data: List[Tuple[str, int]], 
    scale_min: float = 0, 
    scale_max: float = 100
) -> List[Tuple[str, float]]:
    """
    Perform Z-score normalization with optional scaling
    
    Args:
        data (List[Tuple[str, int]]): List of tuples with (name, value)
        scale_min (float): Minimum of scaling range
        scale_max (float): Maximum of scaling range
    
    Returns:
        List[Tuple[str, float]]: List of tuples with scaled normalized values
    """
    # Extract values for normalization
    values = [item[1] for item in data]
    
    # Calculate Z-score
    mean = np.mean(values)
    std = np.std(values)
    
    # Handle edge case of zero standard deviation
    if std == 0:
        normalized_values = [scale_min] * len(values)
    else:
        # Z-score calculation
        z_scores = [(x - mean) / std for x in values]
        
        # Optional scaling to desired range
        normalized_values = [
            scale_min + (zscore - min(z_scores)) * (scale_max - scale_min) / 
            (max(z_scores) - min(z_scores))
            for zscore in z_scores
        ]
    
    # Reconstruct the list of tuples with normalized values
    return [(name, norm_value) for (name, _), norm_value in zip(data, normalized_values)]


In [ ]:
normalized_data = z_score_normalize_with_scaling(all_passes)

In [ ]:
for original, normalized in zip(all_passes, normalized_data):
        print(f"{original} -> {normalized}")

In [ ]:
from collections import defaultdict

def combine_player_stats(*metrics: List[Tuple[str, int]], default_value: int = 0) -> Dict[str, List[int]]:
    """
    Combine multiple lists of player statistics, ensuring consistent player names
    and filling missing values with a default.
    
    Args:
        *lists: Variable number of lists containing (player_name, value) tuples
        default_value: Value to use when a player is missing from a list
    
    Returns:
        Dictionary with player names as keys and lists of values from each input list
    """
    # Collect all unique player names
    all_players = set()
    for metric in metrics:
        all_players.update(player for player, _ in metric)
    
    # Create a result dictionary with default structure
    result = {player: [default_value] * len(metrics) for player in all_players}
    
    # Populate the dictionary
    for idx, metric in enumerate(metrics):
        for player, value in metric:
            result[player][idx] = value
    
    return result


In [ ]:
normalized_all_passes = z_score_normalize_with_scaling(all_passes)
normalized_successful_passes = z_score_normalize_with_scaling(successful_passes)
normalized_key_passes = z_score_normalize_with_scaling(key_passes)
normalized_long_passes = z_score_normalize_with_scaling(long_passes)
normalized_shots_on_goal = z_score_normalize_with_scaling(shots_on_goal)
normalized_all_aerial_duels = z_score_normalize_with_scaling(all_aerial_duels)
normalized_successful_aerial_duels = z_score_normalize_with_scaling(successful_aerial_duels)


In [ ]:
normalized_player_data = combine_player_stats(normalized_all_passes, normalized_successful_passes, normalized_key_passes, normalized_long_passes, normalized_shots_on_goal, normalized_all_aerial_duels, normalized_successful_aerial_duels)

In [ ]:
normalized_player_data

In [ ]:
players_data = combine_player_stats(all_passes, successful_passes, key_passes, long_passes, shots_on_goal, all_aerial_duels, successful_aerial_duels)

In [ ]:
all_passes_dict =  [{pas[0]: pas[1]} for pas in all_passes]

In [ ]:
all_passes_dict

In [ ]:
players_data

In [ ]:
all_passes

In [ ]:
import pandas as pd

In [ ]:
METRICS = [
    "Number of Passes",
    "Successful Passes",
    "Number of Key Passes",
    "Number of Long Passes",
    "Number of Shots on Goal",
    "Number of Aerial Duels",
    "Aerial Duel Success",
]

In [ ]:

player = pd.DataFrame(players_data, index=METRICS).T

In [ ]:
player

In [ ]:
type(player)

In [ ]:
all_passes

In [ ]:
normalized_data = z_score_normalize_with_scaling(all_passes)

In [ ]:
normalized_data